<a href="https://colab.research.google.com/github/wangjb/cwb_open_data/blob/master/rt_meteo_info_unmanned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def retrieve_data(datacode,auth):
  import urllib
  # query string to CWB open data service
  url='https://opendata.cwb.gov.tw/'
  sub_path='fileapi/v1/opendataapi/'
  # instant 10 min precip
  datalimit=''
  dataoffset=''
  dataformat='JSON'
  stationID=''
  datastatus=''

  # create query string
  url_path=url+sub_path+datacode
  query_fields=['Authorization','limit','offset','format','stationID','status']
  query_data=(auth,datalimit,dataoffset,dataformat,stationID,datastatus)
  query_string=dict(zip(query_fields,query_data))

  # request data
  import requests
  r = requests.get(url_path,params=query_string,stream=True)
  jsondata = r.json()

  return jsondata

In [2]:
datacode = 'O-A0001-001'
auth = None
jsondata = retrieve_data(datacode, auth)

In [3]:
jsondata['cwbopendata']['location'][0].keys()

dict_keys(['lat', 'lon', 'lat_wgs84', 'lon_wgs84', 'locationName', 'stationId', 'time', 'weatherElement', 'parameter'])

In [4]:
import pandas as pd
data = pd.DataFrame(jsondata['cwbopendata']['location'])

# process obstime
data['obstime'] = pd.DataFrame(data['time'].to_list())

# process weatherElement
weatherElement = pd.DataFrame(data['weatherElement'].to_list())
weatherElement.columns = [ x['elementName'] for x in weatherElement.iloc[0] ]
weatherElement = weatherElement.applymap(lambda x: x['elementValue']['value'])

# process parameter
parameter = pd.DataFrame(data['parameter'].to_list())
parameter.columns = [ x['parameterName'] for x in parameter.iloc[0] ]
parameter = parameter.applymap(lambda x: x['parameterValue'])

# merge data
data = data.merge(weatherElement,left_index=True,right_index=True)
data = data.merge(parameter,left_index=True,right_index=True)
data = data.drop(['time','weatherElement','parameter'],axis=1)

In [5]:
data

,lat,lon,lat_wgs84,lon_wgs84,locationName,stationId,obstime,ELEV,WDIR,WDSD,TEMP,HUMD,PRES,H_24R,H_FX,H_XD,H_FXT,D_TX,D_TXT,D_TN,D_TNT,CITY,CITY_SN,TOWN,TOWN_SN
0,25.035950,121.611456,25.0341638888889,121.619680555556,國三南深路交流道,CM0010,2020-08-12T16:00:00+08:00,68.0,238,0.3,27.5,0.75,1006.0,0.0,-99,-99,-99,34.80,2020-08-12T13:10:00+08:00,24.80,2020-08-12T05:30:00+08:00,臺北市,01,南港區,035
1,24.091036,120.428836,24.0892916666667,120.436986111111,水試所鹿港,CM0110,2020-08-12T16:00:00+08:00,7.0,281,2.8,30.3,0.72,1012.2,-99,-99,-99,-99,32.10,2020-08-12T11:50:00+08:00,26.20,2020-08-12T05:50:00+08:00,彰化縣,14,鹿港鎮,140
2,23.718408,120.183736,23.7166666666667,120.191686111111,水試所臺西,CM0120,2020-08-12T16:00:00+08:00,4.0,233,5.2,30.4,0.73,1012.0,0.0,-99,-99,-99,31.10,2020-08-12T14:20:00+08:00,26.20,2020-08-12T05:40:00+08:00,雲林縣,16,臺西鄉,187
3,23.122550,120.071589,23.1208027777778,120.079675,水試所七股,CM0130,2020-08-12T16:00:00+08:00,11.0,187,7.3,30.7,0.72,1011.4,0.0,-99,-99,-99,31.60,2020-08-12T14:30:00+08:00,27.10,2020-08-12T06:10:00+08:00,臺南市,04,七股區,253
4,23.101389,121.375261,23.099575,121.383358333333,水試所成功,CM0140,2020-08-12T16:00:00+08:00,26.0,214,1.3,31.6,0.68,1011.5,0.0,-99,-99,-99,31.90,2020-08-12T14:50:00+08:00,26.10,2020-08-12T05:20:00+08:00,臺東縣,22,成功鎮,258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,23.300847,120.656664,23.299075,120.664766666667,茶山,C0M800,2020-08-12T16:00:00+08:00,440.0,0,0.2,27.9,0.79,964.0,5.5,10.2,222,2020-08-12T15:07:00+08:00,32.20,2020-08-12T13:50:00+08:00,21.30,2020-08-12T05:30:00+08:00,嘉義縣,17,阿里山鄉,208
429,23.688211,120.595289,23.68645,120.603416666667,棋山,C0K560,2020-08-12T16:00:00+08:00,117.0,196,0.6,26.3,0.91,1000.7,3.0,-99,-99,-99,33.90,2020-08-12T11:20:00+08:00,23.60,2020-08-12T05:20:00+08:00,雲林縣,16,古坑鄉,196
430,23.516064,120.221625,23.5143194444444,120.229736111111,蔦松,C0K550,2020-08-12T16:00:00+08:00,15.0,252,0.9,30.3,0.78,1010.7,0.0,-99,-99,-99,33.50,2020-08-12T12:50:00+08:00,25.50,2020-08-12T05:40:00+08:00,雲林縣,16,水林鄉,204
431,23.269517,120.117367,23.2677722222222,120.125480555556,北門,C0X290,2020-08-12T16:00:00+08:00,10.0,220,5.8,31.0,0.73,1010.8,0.0,-99,-99,-99,32.20,2020-08-12T12:30:00+08:00,26.20,2020-08-12T05:40:00+08:00,臺南市,04,北門區,237
